In [1]:
import numpy as np
import torch
from torch import nn
import math
from pprint import pprint
import pandas as pd
from tqdm.notebook import tqdm
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
train = pd.read_csv("./train.csv")
test_comments = pd.read_csv("./test.csv")
test_labels = pd.read_csv("./test_labels.csv")

In [3]:
test = pd.merge(test_comments, test_labels, on="id")

In [5]:
import nltk
from nltk import word_tokenize
from nltk.util import ngrams

In [12]:
columns = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

In [24]:
counts = dict()
for i in range(len(train)):
    tokens = nltk.word_tokenize(train.comment_text[i])
    targets = train.loc[i][columns].to_numpy()
    if i % 1000 == 0:
        print(i)
    for token in tokens:
        if token not in counts:
            counts[token] = np.zeros(6)
        counts[token] = counts[token] + targets

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [20]:
train[columns].loc[100]

toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
Name: 100, dtype: int64

In [27]:
words = []
probs = []
for i, (k, v) in enumerate(counts.items()):
    words.append(k)
    probs.append(v)

In [33]:
p = np.concatenate([prob.reshape(1, -1) for prob in probs], axis=0)
p

array([[1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
       [726.0, 60.0, 342.0, 9.0, 292.0, 53.0],
       [16845.0, 875.0, 8372.0, 312.0, 7108.0, 1177.0],
       ...,
       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]], dtype=object)

In [35]:
p = p / len(train)

In [45]:
top = np.argsort(p, axis=0)[:500:-1]

for i in range(50):
    print(top[i])


[45 45 45 45 45 45]
[ 31 182  31  24  31  31]
[ 17  31  17  31 137  17]
[  137    17   137    17    17 28368]
[ 24 137 110 137 110 137]
[  110    24    24 29247    24   110]
[   2 1602    2 1558    2   74]
[  67 1172   79  182  182   24]
[   79   110    67    67    79 14036]
[  112  9881   182  4806    67 72130]
[  74 1558  112   79   74   79]
[ 84 167  74 167  84   2]
[  13 7430   84  420  420   67]
[ 182 1177 1602    2  112 1121]
[  420    79   420 17521   167 29247]
[  72  420   13  110 1602   84]
[   93    74   167   418    13 20547]
[ 143 4382  143  804  123  112]
[  167 29247    72    84    72   123]
[ 123 4120   93 6794  143   13]
[ 125 6408 1172  197   93 1602]
[  15  144  123 1173 1172  167]
[   20   540   125    13    15 16788]
[1602    2   15 6405  125   72]
[  34  512    6 2034 9881   15]
[   81    84    34 99431 28368   420]
[  140    67    20 50848     6   125]
[    6  7967  9881 40761    34   182]
[  138   186   140 40760    20   143]
[   65  6683  1037 20797  1037 37788

In [47]:
words[182]

'YOU'

In [48]:
p[182]

array([0.053737834568937964, 0.034404747729850665, 0.04121049564143861,
       0.0035532772245583473, 0.04249519022880097, 0.002644590809106918],
      dtype=object)

In [50]:
label_probs = np.sum(train[columns].to_numpy(), axis=0) / len(train)
label_probs

array([0.09584448, 0.00999555, 0.05294822, 0.00299553, 0.04936361,
       0.00880486])